In [ ]:
import sys
import os
from src import app
from gevent import monkey,pywsgi
CURRENT_DIR = os.getcwd()
sys.path.insert(0, CURRENT_DIR)

In [ ]:
#monkey.patch_all(thread=False,select=False)
server = pywsgi.WSGIServer(('127.0.0.2', 5000), app)
#app.debug = Trueee
server.serve_forever()
#http://127.0.0.2:5000/

In [ ]:
from jobs import job_sl_bonds
from db_drivers import mongo_manager
cfg={"driver": "mongo", "db_name": "trade", "host": "localhost",
     "port": 27017, "user": "", "user_pswd": "", "mongo_data": "C:\\Users\\DB"}

a=job_sl_bonds(cfg)

In [1]:
%%time
from sys import path
from datetime import datetime
import os
path.append(os.path.join(os.getcwd(),'src'))

from commons import convert_file_size,init_db_manager
import pandas as pd
from jobs import *
#a=job_world_fond_indexes(init_db_manager())

ModuleNotFoundError: No module named 'jobs.commons'

In [ ]:
a=job_sectors(init_db_manager())

In [ ]:
db=init_db_manager()
a=db.get_table('mosex_securities',result='json')
#a=job_update_sec_hist(init_db_manager())

In [ ]:
a=pd.DataFrame(a)

In [ ]:
a.shape

In [ ]:
db.get_table('sec_history_manager_mosex')

In [ ]:
db.get_table('mosex_sec_history')[-10:]

In [ ]:
r=db.find_one('sec_history_manager_mosex',{"secid":'GAZP111'})

In [ ]:
from datetime import datetime
datetime.strptime('2016-01-04','%Y-%m-%d').month

In [ ]:
db=init_db_manager()

In [ ]:
df[df.boardid=='TQBR'].tradedate.nunique()

In [ ]:
from sys import path
path.append(r"C:\Users\Kart\jupyter_projects\webtrade\webtrade\external_sources")

from external_sources import mosex,tradingview
import pandas as pd
a=mosex()
#a=tradingview()

In [ ]:

aa=a.security_hist('SBER')

In [ ]:
aa[1][1]

In [ ]:
datetime.now()

In [ ]:
for el in list(map(lambda s:s.lower(),aa[1][0])):
    print(el+"="+el+"_ind,")

In [ ]:
aa[1][0]

In [ ]:
df=aa[1][1]

In [ ]:
aa.json()['history.cursor']

In [ ]:
%%time
a=job_sectors(init_db_manager())

In [ ]:
from requests import Session
from multiprocessing.dummy import Pool as ThreadPool
s = Session()
s.get("http://ya.ru")
def f(url):
    return s.get(url)


    
urls=[r'https://www.rbc.ru/',r'http://ya.ru/']

In [ ]:
pool = ThreadPool(4)
results=pool.map_async(f,urls)
pool.close()
pool.join()

In [ ]:
results

In [ ]:
def job_update_sec_hist(db_manager)->tuple:
    res,raw_data=mosex().security_hist('SBER')
    if res is False:
        return (res,)
    
    return raw_data

In [ ]:
raw_data=a

In [ ]:
columns=raw_data[0]
raw_data=raw_data[1]

In [ ]:
columns

In [ ]:
import pandas as pd
pd.DataFrame(raw_data[-1],columns=columns)